# Collecting Teacher LLM Answers and Rationales

## Introduction
In this notebook, we will collect answers and rationales from the Llema and GPT-Neo teacher LLMs. The questions previously collected and tokenized via data_preprocessing_aqua_questions.py will be used. The answers and rationales will be collected, a small sample will be checked for correctness and brevity, then the full sample will be collected.

In [1]:
import pandas as pd
import numpy as np
from transformers import T5Tokenizer, AutoTokenizer, AutoModelForCausalLM, generate
import torch
import matplotlib.pyplot as plt
import os

Set the file path, tokenizers, and models

In [2]:
directory = 'data/preprocessed/'
filename = 'preprocessed_aqua_data.pt'
preprocessed_file_path = os.path.join(directory, filename)

t5_tokenizer = T5Tokenizer.from_pretrained('google-t5/t5-small')
llemma_tokenizer = AutoTokenizer.from_pretrained("EleutherAI/llemma_7b")
gptneo_tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-2.7B")

llema_model_name = "EleutherAI/llemma_7b"
llema_model = AutoModelForCausalLM.from_pretrained(llema_model_name)

gptneo_model_name = "EleutherAI/gpt-neo-2.7B"
gptneo_tokenizer = AutoTokenizer.from_pretrained(gptneo_model_name)

Move model to GPU

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
llema_model.to(device)

Load data

In [ ]:
try:
    preprocessed_data = torch.load(preprocessed_file_path)
    print("Preprocessed data loaded successfully!")
except FileNotFoundError:
    print(f"File not found at {preprocessed_file_path}. Please check the path and try again.")

Inspect data

In [ ]:
example_teacher_input = preprocessed_data['train']['teacher_inputs']['llemma'][0]
train_teacher_input_llema = preprocessed_data['train']['teacher_inputs']['llemma']
train_teacher_input_gptneo = preprocessed_data['train']['teacher_inputs']['gptneo']
train_student_inputs = preprocessed_data['train']['student_inputs']

assert len(train_teacher_input_llema) == len(train_teacher_input_gptneo) == len(train_student_inputs), "Data lengths do not match!"

print(f"Number of training questions for Llema teacher LLMs: {len(train_teacher_input_llema)}")
print(f"Number of training questions for GPT-Neo teacher LLMs: {len(train_teacher_input_gptneo)}")
print(f"Number of training questions for student LLM: {len(train_student_inputs)}")

decoded_question = llemma_tokenizer.decode(train_teacher_input_llema[0]['input_ids'].squeeze(), skip_special_tokens=True)
print("Example of a tokenized question for Llemma:")
print(decoded_question)

Set the model to evaluation mode since we're just generating responses

In [ ]:
llema_model.eval()
print("Llemma model loaded successfully!")

Generate responses for each question

In [ ]:
# Iterate over the tokenized questions
llema_responses = []

for i, input_data in enumerate(train_teacher_input_llema):
    # Move the input tensor to the correct device (GPU or CPU)
    input_ids = input_data['input_ids'].to(device)
    attention_mask = input_data['attention_mask'].to(device)
    
    # Generate output using the model
    output = llema_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=512,  # Limit response length
        num_return_sequences=1,  # Only generate one response per question
        do_sample=True,  # Sampling to add randomness for more diverse responses
        top_p=0.9,  # Top-p sampling for more controlled outputs
        temperature=0.7  # Temperature parameter to adjust randomness
    )
    
    # Decode the generated output
    response_text = llemma_tokenizer.decode(output[0], skip_special_tokens=True)
    
    # Append the response to the list
    llema_responses.append(response_text)
    
    # Optional: Print progress every 10 questions
    if i % 10 == 0:
        print(f"Processed {i + 1} questions")

Parse responses, extract answer & rationale

In [ ]:
# Example of parsing the response to separate answer and rationale
answers_rationales = []

for response in llema_responses:
    # Assume the model responds with: "Answer: ... Rationale: ..."
    try:
        answer_start = response.index("Answer: ") + len("Answer: ")
        rationale_start = response.index("Rationale: ") + len("Rationale: ")
        answer = response[answer_start:rationale_start].strip()
        rationale = response[rationale_start:].strip()
        
        answers_rationales.append({
            'answer': answer,
            'rationale': rationale
        })
    except ValueError:
        # Handle case where the response does not follow the expected format
        answers_rationales.append({
            'answer': "Could not parse answer",
            'rationale': response
        })
